# Backend for the final project!

In this notebook, we will learn a few things.
- Quick async and asyncio introduction
- How to load API keys
- How to setup FastAPI endpoints
- How to make models with Pydantic for data validation
- How to launch the uvicorn server

*FOR THE BEST EXPERIENCE, IT'S RECOMMENDED AND ENCOURAGED NOT TO RUN CODE IN THE NOTEBOOK, BUT COPY THE CODE AND RUN IT IN A USUAL ```.py``` FILE*

## 🚀 Asynchronous programming
Understanding async and await in Python:
- async and await are used for writing asynchronous (non-blocking) code.
- Instead of waiting for one task to finish before starting another, async functions run concurrently.
- This is useful for I/O-bound tasks like API requests, database queries, and file operations.

Let's consider a few examples to get a grasp of it:

In [ ]:
# By default, all code in python is synchronous and blocking.
# You cannot execute the next line of code unless the previous one is completed

import time

def task():
    print("Task started")
    time.sleep(2)  # Simulating a delay
    print("Task completed")

print("Before task")
task()
print("After task")


In [ ]:
import asyncio

# here we define two separate tasks and launch them together
# they run concurrently
# (this cell will not run from the .ipynb)

async def task1():
    print("Task 1 started")
    await asyncio.sleep(2)
    x = "Task 1 result"
    print("Task 1 completed")
    return await x

async def task2():
    print("Task 2 started")
    await asyncio.sleep(1)
    x = "Task 2 result"
    print("Task 2 completed")
    return await x

async def main():
    await asyncio.gather(task1(), task2())  # Run both tasks in parallel

# async functions have to be awaited - thats where we use await keyword
# async functions can run within main code loop but they will not return what we want, because they were not awaited
# we can use asyncio library to await them in a synchronous context

main() # - will not return print statements
asyncio.run(main()) # - will resolve normally

This will become very important once we get to FastAPI, a library that allows us to build endpoints that can process user requests in parallel

## 📌 API Key Management
API keys are stored in an `.env` file and loaded using `dotenv`. This ensures security by keeping keys outside the codebase. They are used to access API endpoints of different services. The provider gives you access to the resources with the keys.

Usually, you would have to setup the API keys as your machine's environment variables. ```dotenv``` library helps us do it the fastest way. But first we need to put all our keys into a ```.env``` file.

In [ ]:
import os
from dotenv import load_dotenv

dotenv_path = os.path.join(os.path.dirname(__file__), '.env')
load_dotenv(dotenv_path=dotenv_path, override=True)

print(os.environ['OPENAI_API_KEY'])  # Ensure API key is loaded
print(os.environ['TAVILY_API_KEY'])

## 🚀 Backend API with FastAPI


### Why FastAPI?
FastAPI is a modern, fast (high-performance) web framework for building APIs with Python 3.7+, based on type hints.

It is built on a few principles:
- Data Validation: you need to define the data models of inbound and outbound requests
- Automatic: built in Swagger UI to test your endpoints and OpenAPI format documentation
- Easy-to-Use: you can define your endpoints and you are basically ready to perform operations

## FastAPI Setup
FastAPI allows us to define API endpoints easily. First, we instantiate the app:

In [ ]:
from fastapi import FastAPI
app = FastAPI()  # Create FastAPI instance

This is our main body of the backend. Further we are going to build an endpoint and define request and response models.

## 📌 Request Models with Pydantic
FastAPI uses Pydantic for data validation.
We need to define models for requests and responses.
This ensures we always receive and send data that is predictable.
Pydantic is a library just made for that!
It allows FastAPI to dynamically validate inbound and outbound requests.

In [ ]:
from pydantic import BaseModel

# Here we define the user request, which will be a string
class QueryRequest(BaseModel):
    prompt: str

# This will be our response model
class QueryResponse(BaseModel):
    response: str

We can access attributes of the data model as a class attribute: ```QueryRequest.prompt```

## 📌 Defining API Endpoints
We now define endpoints for querying the LLM and setting up API keys. For that we need to define the URL of our endpoint.

Consider this function.

In [ ]:
async def simple_response(prompt: str):
    return {"response": prompt}

Its a simple dummy function to return the incoming prompt back. Notice the return format: it has a key ```response``` just as QueryResponse model.
Now we can actually make a working endpoint for our function.

In [ ]:
@app.post('/query', response_model=QueryResponse)
async def simple_query(payload: QueryRequest):
    response = await simple_response(payload.prompt)
    return response

Notice the ```response_model``` parameter and what we return. Given that FastAPI has internal data validation it will throw exceptions if we try to send a request to our endpoint with data that doesn't fit our validation model, as well as FastAPI will not let us send data that doesn't fit our ```response_model```.

## 📌 Running the API
To actually access our endpoints, we need to launch a server locally. One of such servers is uvicorn, which is often paired with FastAPI.
Run the following command in the terminal to start the FastAPI server using uvicorn:
```sh
uvicorn main:app --reload
```
Visit `127.0.0.1:8000/docs` to test the endpoints interactively.